In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

In [17]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [18]:
tw = line.twiss()
tt = line.get_table()

In [23]:
sliced = tt.rows['.*\..1.*'].name

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/var/folders/b5/swvf1qzj6l36gcd1zzs4_g8h0000gn/T/ipykernel_26682/2654132670.py:1: SyntaxWarning: invalid escape sequence '\.'
  sliced = tt.rows['.*\..1.*'].name


In [27]:
mask = np.array(['mb' not in name for name in sliced])

In [29]:
print(sliced[mask])

['drift_0..1' 'drift_1..1' 'drift_2..1' ... 'drift_mdh.63607..1..1'
 'drift_mdh.63607..2..1' 'bph.63608..1']


In [30]:
len(sliced[mask])

5798

In [32]:
sliced[mask][:50]

array(['drift_0..1', 'drift_1..1', 'drift_2..1', 'drift_3..1',
       'drift_4..1', 'vvsa.10103..1', 'drift_5..1', 'drift_6..1',
       'drift_mdv.10107..1..0', 'drift_mdv.10107..1..1',
       'drift_mdv.10107..2..1', 'bpv.10108..1', 'drift_8..1',
       'drift_9..1', 'drift_10..1', 'drift_11..1', 'drift_12..1',
       'drift_13..1', 'drift_14..1', 'drift_mdh.10207..1..0',
       'drift_mdh.10207..1..1', 'drift_mdh.10207..2..1', 'bph.10208..1',
       'drift_16..1', 'drift_17..1', 'drift_18..1', 'drift_19..1',
       'drift_20..1', 'drift_21..1', 'drift_22..1', 'drift_23..1',
       'drift_mdv.10307..1..0', 'drift_mdv.10307..1..1',
       'drift_mdv.10307..2..1', 'bpv.10308..1', 'drift_25..1',
       'drift_26..1', 'drift_27..1', 'drift_28..1', 'drift_29..1',
       'drift_30..1', 'drift_31..1', 'drift_mdh.10407..1..0',
       'drift_mdh.10407..1..1', 'drift_mdh.10407..2..1', 'bph.10408..1',
       'drift_33..1', 'drift_34..1', 'drift_35..1', 'drift_36..1'],
      dtype=object)

In [36]:
idx = np.where(tw.name == 'drift_mdh.10407..1..0')[0][0]
for name in tw.name[idx-10:idx+10]:
    print(name)

drift_30..1
vttb.10402.a_aper
drift_30..2
vttb.10402.z_aper
vttb.10402.b_aper
drift_30..3
loe.10402
drift_31..0
vttb.10402.c_aper
drift_31..1
drift_mdh.10407..1..0
mdh.10407.a_aper
drift_mdh.10407..1..1
mdh.10407
drift_mdh.10407..2..0
mdh.10407.b_aper
drift_mdh.10407..2..1
drift_32
bph.10408..0
bph.10408.a_aper


In [37]:
line['mdh.10407']

View of Multipole(_order=np.int64(0), inv_factorial_order=1, length=0.25, hxl=0, radiation_flag=np.int64(0), delta_taper=0, knl=array([1.39562695e-08]), ksl=array([0.]), _isthick=np.int64(0), num_multipole_kicks=np.int64(0), _model=np.int64(0), _integrator=np.int64(0), _sin_rot_s=-999, _cos_rot_s=-999, _shift_x=0, _shift_y=0, _shift_s=0, _rot_x_rad=0, _rot_y_rad=0, _rot_s_rad_no_frame=0, rot_shift_anchor=0, _internal_record_id=RecordIdentifier(buffer_id=np.int64(0), offset=np.int64(0)))

## Fixing line

In [43]:
import shutil
import xtrack as xt
from pathlib import Path

In [298]:
acc_models_dir = Path('/Users/lisepauwels/sps_simulations/acc-models-sps')

In [309]:
replace_mba_mbb_aper_with_rect = True

# Load lattice
line = xt.Line.from_json(acc_models_dir / 'xsuite/sps.json')

# read aperture file
with open(acc_models_dir / 'aperture_ldb/APERTURE_EYETS 2024-2025.seq', 'r') as f:
    mad_aper_content = f.read()

# PATCH: remove installation with wrong location
mad_aper_content = mad_aper_content.replace(
    "install, element = VEQF.10010.A, at= -0.095;",
    "!install, element = VEQF.10010.A, at= 0e-3;")

# Load aperture markers in a dummy sequence
from cpymad.madx import Madx
mad = Madx()
mad.input('''
SPS : SEQUENCE, refer = centre,    L = 7000;
a: marker, at = 20;
endsequence;
''')
mad.input(mad_aper_content)
mad.beam()
mad.use('SPS')
line_aper = xt.Line.from_madx_sequence(mad.sequence.SPS, install_apertures=True)

# Identify the aperture markers
tt_aper = line_aper.get_table().rows['.*_aper']

# PATCH: Aperture markers with known issues
aper_blacklist = [
    'vtaf.51632.b_aper', 'vbrta.51633.a_aper', 'vbrta.51633.b_aper',
    'bgiha.51634.a_aper', 'bgiva.51674.a_aper']


Loading line from dict:   0%|          | 0/4482 [00:00<?, ?it/s]

Done loading line from dict.           

  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2025.09.17 19:15:23      +
  ++++++++++++++++++++++++++++++++++++++++++++
++++++ info: seqedit - number of elements installed:  5061
++++++ info: seqedit - number of elements moved:      0
++++++ info: seqedit - number of elements removed:    0
++++++ info: seqedit - number of elements replaced:   0


Converting sequence "sps":   0%|          | 0/8557 [00:00<?, ?it/s]

In [302]:
base, number, aper_type = tt_aper.name[0].split('.')

In [304]:
tt_aper.rows[f'{base}.{number}.*']

Table: 1 row, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
veqf.10010.b_aper         3.251 LimitEllipse   False     False None               False

In [310]:
env = line.env
insertions = []
for nn in tt_aper.name:
    if nn in aper_blacklist:
        continue

    elem = line_aper.get(nn).copy()

    # PATCH: Replace MBA and MBB apertures with rectangular ones (closer to reality)
    # NB: the aperture in LDB are still the aperture without compressions
    # real apertures are smaller
    if replace_mba_mbb_aper_with_rect:
        if nn.startswith('veba') or nn.startswith('vebb'):
            assert isinstance(elem, xt.LimitEllipse)
            new_elem = xt.LimitRect(min_x=-elem.a, max_x=elem.a,
                                    min_y=-elem.b, max_y=elem.b)
            elem = new_elem

    env.elements[nn] = elem
    base, number, aper_type = nn.split('.')
    ll = tt_aper.rows[f'{base}.{number}.*']
    if len(ll)>1:
        insertions.append(env.place(nn, at=tt_aper['s', nn]))
    else:
        if aper_type == 'aper_a':
            insertions.append(env.place(nn, at=tt_aper['s', nn] + '@start'))
        elif aper_type == 'aper_b':
            insertions.append(env.place(nn, at=tt_aper['s', nn] + '@end'))

In [311]:
# PATCH: Shorten all pipes by 1 mm to avoid overlaps (to be removed when
# the LDB model is fixed)
# some pipes have also a c, to be refined.
for ins in insertions:
    if ins.name.endswith('.a_aper'):
        ins.at += 0.6e-3
    if ins.name.endswith('.b_aper'):
        ins.at -= 0.6e-3

# Insert the apertures into the line
line.insert(insertions, s_tol=1e-6)

# Associate main quadrupole and dipole chambers with the corresponding magnets
for nn in line.element_names:
    if nn.startswith('qf.') or nn.startswith('qd.'):
        name_aper = 've' + nn + '.b_aper'
        assert name_aper in line.env.elements
        line[nn].name_associated_aperture = name_aper
    elif nn.startswith('mbb.') or nn.startswith('mbe.'):
        name_aper = 've' + nn[1:] + '.b_aper'
        if nn == 'mbb.20150':
            name_aper = 'vebb.20151.b_aper' # PATCH: This aperture has a strange naming
        assert name_aper in line.env.elements
        line[nn].name_associated_aperture = name_aper

# PATCH: arc bellows miss one profile
tt = line.get_table()
insertions = []
for kk in ['b', 'c', 'd', 'v']:
    tt_patch = line.get_table().rows[f'vtt{kk}\..*\.a_aper']
    for nn in tt_patch.name:
        nn_new = nn.replace('.a_aper', '.z_aper')
        env.elements[nn_new] = line_aper.get(nn).copy()
        insertions.append(env.place(nn_new,
                                    at=nn.replace('.a_aper', '.b_aper') + '@start'))
line.insert(insertions, s_tol=1e-6)

<>:30: SyntaxWarning: invalid escape sequence '\.'
<>:30: SyntaxWarning: invalid escape sequence '\.'
/var/folders/b5/swvf1qzj6l36gcd1zzs4_g8h0000gn/T/ipykernel_26682/3946787449.py:30: SyntaxWarning: invalid escape sequence '\.'
  tt_patch = line.get_table().rows[f'vtt{kk}\..*\.a_aper']


Slicing line:   0%|          | 0/4482 [00:00<?, ?it/s]

Slicing line:   0%|          | 0/15045 [00:00<?, ?it/s]

In [312]:
tt = line.get_table()

In [313]:
tt.rows['.*tidp.*']

Table: 4 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434.a_aper        453.64 LimitRect      False     False None               False
tidp.11434..0           453.641 DriftSlice      True     False tidp.11434         False
tidp.11434.b_aper       457.939 LimitRect      False     False None               False
tidp.11434..1           457.939 DriftSlice      True     False tidp.11434         False

# Fixing collimators

In [177]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [153]:
for nn in env.elements:
    if env.elements[nn].__class__.__name__ == 'DriftSlice':
        print(nn)
        env.elements[nn] = xt.Drift(length= env.elements[nn]._parent.length * env.elements[nn].weight)

drift_0..0
drift_0..1
drift_0..2
drift_1..0
drift_1..1
drift_1..2
drift_2..0
drift_2..1
drift_2..2
drift_3..0
drift_3..1
drift_3..2
drift_4..0
drift_4..1
drift_4..2
drift_4..3
vvsa.10103..0
vvsa.10103..1
vvsa.10103..2
drift_5..0
drift_5..1
drift_5..2
drift_5..3
drift_6..0
drift_6..1
drift_mdv.10107..1..0
drift_mdv.10107..1..1
drift_mdv.10107..2..0
drift_mdv.10107..2..1
bpv.10108..0
bpv.10108..1
bpv.10108..2
drift_8..0
drift_8..1
drift_9..0
drift_9..1
drift_9..2
drift_10..0
drift_10..1
drift_10..2
drift_11..0
drift_11..1
drift_11..2
drift_12..0
drift_12..1
drift_12..2
drift_13..0
drift_13..1
drift_13..2
drift_13..3
drift_13..4
drift_13..5
drift_13..6
drift_14..0
drift_14..1
drift_mdh.10207..1..0
drift_mdh.10207..1..1
drift_mdh.10207..2..0
drift_mdh.10207..2..1
bph.10208..0
bph.10208..1
bph.10208..2
drift_16..0
drift_16..1
drift_17..0
drift_17..1
drift_17..2
drift_18..0
drift_18..1
drift_18..2
drift_19..0
drift_19..1
drift_19..2
drift_20..0
drift_20..1
drift_20..2
drift_21..0
drift_21..1

In [178]:
# s_start_tidp = line.get_s_position('tidp.11434..0')
# s_start_tcsm = line.get_s_position('tcsm.51932..0')
line.remove(tt.rows['tidp.*_aper'].name)
line.remove(tt.rows['tcsm.*_aper'].name)

In [135]:
for nn in ['tidp.11434..0', 'tidp.11434..1', 'tidp.11434..2']:
    print(nn)
    env.elements[nn] = xt.Drift(length= env.elements[nn]._parent.length * env.elements[nn].weight)

tidp.11434..0
tidp.11434..1
tidp.11434..2


In [161]:
line.replace('tidp.11434..1', 'tidp.11434', s_tol=1e-1)

In [167]:
tt = line.get_table()

In [168]:
tt.rows['.*tidp.*']

Table: 1 row, 11 cols
name                   s element_type isthick isreplica parent_name iscollective       s_start ...
tidp.11434        453.64 Drift           True     False None               False        453.64

In [175]:
idx = int(np.where(tt.name == 'tidp.11434')[0][0])

In [176]:
tt.name[idx-5:idx+5]

array(['vedu.11432.b_aper', 'drift_115..5', 'vedu.11432.c_aper',
       'drift_115..6', 'drift_1906', 'tidp.11434', 'drift_1907',
       'drift_116..0', 'vbba.11451.a_aper', 'drift_116..1'], dtype=object)

In [179]:
a = line.remove('tidp.11434..0')
b = line.remove('tidp.11434..2')

In [180]:
a

In [151]:
tt

Table: 31557 rows, 11 cols
name                             s element_type        isthick isreplica parent_name iscollective ...
sps$start                        0 Marker                False     False None               False
begi.10010                       0 Marker                False     False None               False
qf.10010                         0 Quadrupole             True     False None               False
drift_0..0                   3.085 Drift                  True     False None               False
veqf.10010.b_aper           3.2504 LimitEllipse          False     False None               False
drift_0..1                  3.2504 Drift                  True     False None               False
veba.10030.a_aper           3.2516 LimitRect             False     False None               False
drift_0..2                  3.2516 Drift                  True     False None               False
mba.10030_entry            3.44499 Marker                False     False None          

In [200]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [201]:
idx = int(np.where(tt.name == 'tidp.11434..1')[0][0])
tt.name[idx-10:idx+5]

array(['vmga.11420.b_aper', 'drift_115..3', 'vedu.11432.a_aper',
       'drift_115..4', 'vedu.11432.b_aper', 'drift_115..5',
       'vedu.11432.c_aper', 'drift_115..6', 'tidp.11434..0',
       'tidp.11434.a_aper', 'tidp.11434..1', 'tidp.11434.b_aper',
       'tidp.11434..2', 'drift_116..0', 'vbba.11451.a_aper'], dtype=object)

In [194]:
s_start_tidp = line.get_s_position('tidp.11434..0')
line.remove(tt.rows['tidp.*'].name)

line.insert(env.place('tidp.11434', at=s_start_tidp))

env.elements['tidp.11434.a_aper'] = xt.LimitEllipse(a=0.1, b=0.1)
env.elements['tidp.11434.b_aper'] = xt.LimitEllipse(a=0.1, b=0.1)

line.insert([env.place('tidp.11434.a_aper', at='tidp.11434' + '@start'),
             env.place('tidp.11434.b_aper', at='tidp.11434' +'@end')])

Slicing line:   0%|          | 0/33038 [00:00<?, ?it/s]

Slicing line:   0%|          | 0/33030 [00:00<?, ?it/s]

In [195]:
tt2 = line.get_table()
idx2 = int(np.where(tt2.name == 'tidp.11434')[0][0])
tt2.name[idx2-5:idx2+5]

array(['veqf.11410.b_aper', 'drift_115..1', 'vmga.11420.a_aper',
       'drift_115..2..0', 'tidp.11434.a_aper', 'tidp.11434',
       'tidp.11434.b_aper', 'drift_1907..1', 'drift_1908', 'drift_116..0'],
      dtype=object)

In [196]:
line.get_s_position('vedu.11432.c_aper')

AssertionError: 

In [197]:
tt2.rows['vedu.11432.*']

Table: 0 rows, 11 cols

In [198]:
tt.rows['vedu.11432.*']

Table: 3 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
vedu.11432.a_aper        452.61 LimitEllipse   False     False None               False
vedu.11432.b_aper        452.81 LimitEllipse   False     False None               False
vedu.11432.c_aper        453.44 LimitEllipse   False     False None               False

In [199]:
tt.rows['.*tidp.*']

Table: 5 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434..0            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434.a_aper        453.64 LimitRect      False     False None               False
tidp.11434..1            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434.b_aper       457.939 LimitRect      False     False None               False
tidp.11434..2           457.939 DriftSlice      True     False tidp.11434         False

# Weird removal of apertures

In [321]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
# tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [204]:
idx = int(np.where(tt.name == 'tidp.11434..1')[0][0])
tt.name[idx-20:idx+5]

array(['bph.11408..2', 'drift_114..0', 'veqf.11410.a_aper',
       'drift_114..1', 'qf.11410', 'drift_115..0', 'veqf.11410.b_aper',
       'drift_115..1', 'vmga.11420.a_aper', 'drift_115..2',
       'vmga.11420.b_aper', 'drift_115..3', 'vedu.11432.a_aper',
       'drift_115..4', 'vedu.11432.b_aper', 'drift_115..5',
       'vedu.11432.c_aper', 'drift_115..6', 'tidp.11434..0',
       'tidp.11434.a_aper', 'tidp.11434..1', 'tidp.11434.b_aper',
       'tidp.11434..2', 'drift_116..0', 'vbba.11451.a_aper'], dtype=object)

In [205]:
line['vmga.11420.a_aper']

View of LimitEllipse(a=0.078, b=0.078)

In [206]:
line['veqf.11410.b_aper']

View of LimitEllipse(a=0.076, b=0.0208)

In [207]:
line['vedu.11432.a_aper']

View of LimitEllipse(a=0.078, b=0.078)

In [208]:
def move_list_element_n_backwards(lst, elems, n):
    for elem in elems:
        idx = lst.index(elem)
        if idx < n or idx >= len(lst):
            raise IndexError(f'Index {idx} of element {elem} out of valid range for moving back {n} steps.')

        elem_check = lst.pop(idx)
        assert elem == elem_check
        lst.insert(idx-n, elem)
    return lst

In [322]:
lst = list(line.element_names)
elems = ['tidp.11434.a_aper', 'tidp.11434..2']
n=1
line.element_names = tuple(move_list_element_n_backwards(lst, elems, n))

In [323]:
tt = line.get_table()

In [324]:
tt.rows['.*tidp.*']

Table: 5 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434.a_aper        453.64 LimitRect      False     False None               False
tidp.11434..0            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434..1            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434..2           457.939 DriftSlice      True     False tidp.11434         False
tidp.11434.b_aper        457.94 LimitRect      False     False None               False

In [325]:
line.insert(env.place('tidp.11434', at='tidp.11434..1@end' ))

Slicing line:   0%|          | 0/33040 [00:00<?, ?it/s]

In [326]:
tt2 = line.get_table()
tt2.rows['.*tidp.*']

Table: 4 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434.a_aper        453.64 LimitRect      False     False None               False
tidp.11434..0            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434..1..0         453.64 DriftSlice      True     False tidp.11434         False
tidp.11434              455.789 Drift           True     False None               False

In [245]:
idx = int(np.where(tt.name == 'tidp.11434..1')[0][0])
tt.name[idx-20:idx+5]

array(['bph.11408..2', 'drift_114..0', 'veqf.11410.a_aper',
       'drift_114..1', 'qf.11410', 'drift_115..0', 'veqf.11410.b_aper',
       'drift_115..1', 'vmga.11420.a_aper', 'drift_115..2',
       'vmga.11420.b_aper', 'drift_115..3', 'vedu.11432.a_aper',
       'drift_115..4', 'vedu.11432.b_aper', 'drift_115..5',
       'vedu.11432.c_aper', 'drift_115..6', 'tidp.11434.a_aper',
       'tidp.11434..0', 'tidp.11434..1', 'tidp.11434..2',
       'tidp.11434.b_aper', 'drift_116..0', 'vbba.11451.a_aper'],
      dtype=object)

In [ ]:
s_start_tidp = line.get_s_position('tidp.11434..0')

line.remove('tidp.11434..0')
line.remove('tidp.11434..1')
line.remove('tidp.11434..2')

line.insert(env.place('tidp.11434', at='tidp.11434..1' ))

In [247]:
tt2 = line.get_table()

In [249]:
idx2 = int(np.where(tt2.name == 'tidp.11434.a_aper')[0][0])
tt2.name[idx2-20:idx2+5]

array(['bph.11408..1', 'bph.11408.b_aper', 'bph.11408..2', 'drift_114..0',
       'veqf.11410.a_aper', 'drift_114..1', 'qf.11410', 'drift_115..0',
       'veqf.11410.b_aper', 'drift_115..1', 'vmga.11420.a_aper',
       'drift_115..2', 'vmga.11420.b_aper', 'drift_115..3',
       'vedu.11432.a_aper', 'drift_115..4', 'vedu.11432.b_aper',
       'drift_115..5', 'vedu.11432.c_aper', 'drift_115..6',
       'tidp.11434.a_aper', 'drift_1906', 'drift_1907', 'drift_1908',
       'tidp.11434.b_aper'], dtype=object)

In [240]:
tt2.rows['.*tidp.*']

Table: 2 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434               451.49 Drift           True     False None               False
tidp.11434.b_aper        457.94 LimitRect      False     False None               False

In [281]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [282]:
tt.rows['.*tidp.*']

Table: 5 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
tidp.11434..0            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434.a_aper        453.64 LimitRect      False     False None               False
tidp.11434..1            453.64 DriftSlice      True     False tidp.11434         False
tidp.11434.b_aper       457.939 LimitRect      False     False None               False
tidp.11434..2           457.939 DriftSlice      True     False tidp.11434         False

In [284]:
s = tt.rows['tidp.11434..1'].s_center

In [285]:
tw = line.twiss()
tidp = line['tidp.11434..1']._parent.copy()
line.discard_tracker()

In [327]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [331]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434..1'], elements=[tidp])
    return tidp

In [332]:
tidp = install_tidp(line)

Slicing line:   0%|          | 0/33040 [00:00<?, ?it/s]

In [333]:
tt = line.get_table()

In [334]:
tt.rows['.*tidp.*']

Table: 4 rows, 11 cols
name                          s element_type      isthick isreplica parent_name iscollective ...
tidp.11434..1            451.49 EverestCollimator    True     False None               False
tidp.11434..1..1         455.79 DriftSlice           True     False tidp.11434         False
tidp.11434.b_aper       457.939 LimitRect           False     False None               False
tidp.11434..2           457.939 DriftSlice           True     False tidp.11434         False

Conclusion: on s'en fou des ouvertures quand on a un collimateur? 

In [335]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta3.json')
env = line.env
tw = line.twiss()
tt = line.get_table()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [336]:
line.replace(['tidp.11434..0', 'tidp.11434..1', 'tidp.11434..2'], 'tidp.11434')

ValueError: Element ['tidp.11434..0', 'tidp.11434..1', 'tidp.11434..2'] cannot be replaced by tidp.11434 because of different lengths.

In [337]:
line.merge_consecutive_drifts()

AttributeError: 'DriftSlice' object has no attribute 'length'

Write a similar function as merge drifts to switch apertures back and merge DriftSlices with parent element